# 데이터불러오기

In [1]:
import pandas as pd
import re
from collections import Counter
import requests
import bs4
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool, Manager

import warnings
warnings.filterwarnings('ignore')

In [10]:
df = pd.read_excel("./gu_list/서울_성동구.xlsx")
print(df.shape)
df

(3007, 7)


,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,성동구,성수동,반상회성수,한식,203990.604123,449639.391644,https://place.map.kakao.com/70035828#comment
1,성동구,금호동,금남정,한식,201832.254147,449613.311209,https://place.map.kakao.com/136507271#comment
2,성동구,용답동,유노초밥,일식,204548.480553,451202.206564,https://place.map.kakao.com/2084182972#comment
3,성동구,성수동,구펠,카페,204083.698877,449567.825065,https://place.map.kakao.com/746075956#comment
4,성동구,성수동,도담순대국,한식,204693.784291,448876.793154,https://place.map.kakao.com/1637210154#comment
...,...,...,...,...,...,...,...
3002,성동구,행당동,연돈볼카츠 왕십리역점,양식,203394.492845,450982.014096,https://place.map.kakao.com/429282793#comment
3003,성동구,성수동,우리술 당당 성수점,카페,205081.754973,448829.982952,https://place.map.kakao.com/562310593#comment
3004,성동구,홍익동,동막골,한식,203119.651823,451392.166539,https://place.map.kakao.com/21418574#comment
3005,성동구,성수동,크레이트커피,카페,204605.330646,449683.103638,https://place.map.kakao.com/1978587681#comment


# 셀레니움과 양송이스프 

In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
from multiprocessing import Pool, Manager
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument("--disable-extensions")
options.add_argument("disable-infobars") 
options.add_argument("window-size=1920x1080") 
options.add_argument("no-sandbox") 
options.add_argument("disable-gpu") 
options.add_argument("--lang=ko_KR") 
options.add_argument( 'user-agent = Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36')

columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=options)
cnt = 0
for i in df['리뷰주소']:
    try:
        driver.get(i)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            row = [names.text, rate.text[0], review.find(name="span").text]
            series = pd.Series(row, index=review_df.columns)
            review_df = review_df.append(series, ignore_index=True)

        for button_num in range(2, 6): 
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)
                cnt += 1
                if cnt % 100 == 0:
                    print(f'현재{cnt}개 진행완료')
    except:
        cnt += 1
        if cnt % 100 == 0:
            print(f'현재{cnt}개 진행완료')       
        pass
            
    
    
driver.close()

현재100개 진행완료
현재200개 진행완료
현재300개 진행완료
현재400개 진행완료
현재500개 진행완료
현재600개 진행완료
현재700개 진행완료
현재800개 진행완료
현재900개 진행완료
현재1000개 진행완료
현재1100개 진행완료
현재1200개 진행완료
현재1300개 진행완료
현재1400개 진행완료
현재1500개 진행완료
현재1600개 진행완료
현재1700개 진행완료
현재1800개 진행완료
현재1900개 진행완료
현재2000개 진행완료
현재2100개 진행완료
현재2200개 진행완료
현재2300개 진행완료
현재2400개 진행완료
현재2500개 진행완료
현재2600개 진행완료
현재2700개 진행완료
현재2800개 진행완료
현재2900개 진행완료
현재3000개 진행완료
현재3100개 진행완료
현재3200개 진행완료
현재3300개 진행완료
현재3400개 진행완료
현재3500개 진행완료
현재3600개 진행완료
현재3700개 진행완료
현재3800개 진행완료
현재3900개 진행완료
현재4000개 진행완료
현재4100개 진행완료
현재4200개 진행완료
현재4300개 진행완료
현재4400개 진행완료
현재4500개 진행완료
현재4600개 진행완료
현재4700개 진행완료
현재4800개 진행완료
현재4900개 진행완료
현재5000개 진행완료
현재5100개 진행완료
현재5200개 진행완료
현재5300개 진행완료
현재5400개 진행완료
현재5500개 진행완료


In [12]:
print('끝!')

끝!


In [13]:
review_df.shape

(5247, 3)

In [14]:
review_df

,name,score,review
0,완산골명가,5,
1,소문난맛함흥냉면전문점,4,
2,소문난맛함흥냉면전문점,3,동네에 있어서 냉면 먹고 싶으면 가볍게 갈 수 있을 것 같아요!
3,소문난맛함흥냉면전문점,4,면이 얇고 부드러운걸 선호 하는데 확실히 맛있었어요. 다만 저는 평양냉면의 슴슴한...
4,소문난맛함흥냉면전문점,5,맛 있음!!!!
...,...,...,...
5242,우리술당당 성수점,5,성수하늘이랑 성수반달 시켰는데 두개 다 깔끔하고 맛있어요. 성수하늘은 크리미한 소다...
5243,우리술당당 성수점,4,집 근처에 전통주 파는데가 없어서 아쉬웠는데 생겨서 너무 좋네요. 젊은 사장님께서 ...
5244,우리술당당 성수점,5,얼마전에 여자친구랑 술을 먹기로했는데 기념일인데 평소에 먹던거 말고 새로운 술을 ...
5245,우리술당당 성수점,5,너무 예뻐서 다먹고나서 사진을 찍었네요 ㅎㅎ;; 일반 주점보다는 부담없이 갈 수 있...


In [15]:
review_df.to_csv('./서울_성동구.csv',encoding='utf-8-sig',index=False)

In [16]:
df_d = pd.read_csv('./서울_성동구.csv')

In [17]:
df_d

,name,score,review
0,완산골명가,5,NaN
1,소문난맛함흥냉면전문점,4,NaN
2,소문난맛함흥냉면전문점,3,동네에 있어서 냉면 먹고 싶으면 가볍게 갈 수 있을 것 같아요!
3,소문난맛함흥냉면전문점,4,면이 얇고 부드러운걸 선호 하는데 확실히 맛있었어요. 다만 저는 평양냉면의 슴슴한...
4,소문난맛함흥냉면전문점,5,맛 있음!!!!
...,...,...,...
5242,우리술당당 성수점,5,성수하늘이랑 성수반달 시켰는데 두개 다 깔끔하고 맛있어요. 성수하늘은 크리미한 소다...
5243,우리술당당 성수점,4,집 근처에 전통주 파는데가 없어서 아쉬웠는데 생겨서 너무 좋네요. 젊은 사장님께서 ...
5244,우리술당당 성수점,5,얼마전에 여자친구랑 술을 먹기로했는데 기념일인데 평소에 먹던거 말고 새로운 술을 ...
5245,우리술당당 성수점,5,너무 예뻐서 다먹고나서 사진을 찍었네요 ㅎㅎ;; 일반 주점보다는 부담없이 갈 수 있...
